In [1]:
%load_ext autoreload
%autoreload 2

In [66]:
import sys
import argparse
import logging
import threading
import time
import signal
import os
from pathlib import Path

from flask import Flask, jsonify, request

from constants import * 

# from library.base_plugin import BasePlugin
from library.config_utils import validate_config, load_yaml_file, write_yaml_file
from library.plugin_manager import PluginManager

In [3]:
# ###############################################################################
# # LOGGING CONFIGURATION
# ###############################################################################

def running_under_systemd():
    """
    A simple heuristic to detect if we're running under systemd.
    If these environment variables are present, systemd likely launched us.
    """
    return ('INVOCATION_ID' in os.environ) or ('JOURNAL_STREAM' in os.environ)


# # Configure the main logger
# logger = logging.getLogger("PaperPi")
# logger.setLevel(logging.INFO)

# # Avoid adding duplicate handlers
# if not logger.hasHandlers():
#     handler = logging.StreamHandler(sys.stdout)
#     formatter = logging.Formatter(fmt=LOG_FORMAT, datefmt=DATE_FORMAT)
#     handler.setFormatter(formatter)
#     logger.addHandler(handler)

#     if running_under_systemd():
#         try:
#             from systemd.journal import JournalHandler
#             handler = JournalHandler()
#         except ImportError:
#             handler = logging.StreamHandler()
#     else:
#         handler = logging.StreamHandler()

#     formatter = logging.Formatter(
#         fmt='%(asctime)s [%(levelname)s] %(message)s',
#         datefmt='%Y-%m-%d %H:%M:%S'
#     )
#     handler.setFormatter(formatter)
#     logger.addHandler(handler)

# # Plugin Manager Logger
# plugin_manager_logger = logging.getLogger("library.plugin_manager")
# plugin_manager_logger.setLevel(logging.INFO)
# plugin_manager_logger.propagate = True

In [4]:
def setup_logging(level=logging.INFO):
    # Set up the root logger
    logger = logging.getLogger()
    logger.setLevel(level)

    # Remove existing handlers to avoid duplicates
    if logger.hasHandlers():
        logger.handlers.clear()

    # Create a console handler
    handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter(fmt=LOG_FORMAT, datefmt=DATE_FORMAT)
    handler.setFormatter(formatter)
    
    # Attach the handler to the root logger
    logger.addHandler(handler)

    # Test logging from the main program
    logger.info("Logger setup complete. Ready to capture logs.")
    
    # Test logging from a simulated library
    library_logger = logging.getLogger("library.plugin_manager")

    return logger

logger = setup_logging()

2025-04-13 18:48:10 [INFO] Logger setup complete. Ready to capture logs.


In [5]:
###############################################################################
# FLASK WEB SERVER
###############################################################################

app = Flask(__name__)

# We'll store a flag indicating the daemon loop is running
daemon_running = True
# We'll also detect if we're in systemd mode or foreground
systemd_mode = running_under_systemd()

@app.route('/')
def home():
    return """
    <h1>Welcome to PaperPi</h1>
    <p>Stub login page or config interface will go here.</p>
    <p>Try POSTing to /stop to halt the daemon.</p>
    """

@app.route('/login')
def login():
    # Stub route for future authentication implementation
    return "Login page (to be implemented)."

@app.route('/stop', methods=['POST'])
def stop_route():
    """
    A web endpoint to stop the daemon thread (and Flask).
    In systemd mode, the service will stop in the background.
    In foreground mode, we print 'stopped: press ctrl+c to exit.'
    """
    global daemon_running
    daemon_running = False
    logger.info("Received /stop request; shutting down daemon and Flask...")

    # Ask Flask's built-in server to shut down
    shutdown_server()

    if not systemd_mode:
        # In foreground mode, let the user know they can Ctrl+C
        logger.info("stopped: press ctrl+c to exit")

    return jsonify({"message": "Stopping daemon..."})

def shutdown_server():
    """
    Trigger a shutdown of the built-in Werkzeug server.
    """
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        logger.warning("Not running with the Werkzeug Server, can't shut down cleanly.")
    else:
        func()

In [6]:
###############################################################################
# DAEMON LOOP
###############################################################################

def daemon_loop():
    """
    The background thread that handles e-paper updates.
    It runs until daemon_running = False.
    """
    logger.info("Daemon loop started.")
    while daemon_running:
        logger.info("display update goes here")
        logger.info('morestuff')
        # In production, you might call a function to update the display here
        time.sleep(5)
    logger.info("Daemon loop stopped.")


# from IPython.display import display, clear_output

# current_image_hash = ''
# plugin_manager.update_cycle()
# try:
#     while True:
#         if current_image_hash != plugin_manager.foreground_plugin.image_hash:
#             current_image_hash = plugin_manager.foreground_plugin.image_hash
#             clear_output(wait=True)        
#             display(plugin_manager.foreground_plugin.image)

#         time.sleep(5)
#         plugin_manager.update_cycle()
# except KeyboardInterrupt:
#     logger.info("Stopped update loop")    

In [7]:
###############################################################################
# SIGNAL HANDLING
###############################################################################

def handle_signal(signum, frame):
    """
    Handle SIGINT (Ctrl+C) or SIGTERM (systemctl stop) for a graceful shutdown:
      - Stop the daemon loop
      - Shut down Flask if possible
    """
    logger.info(f"Signal {signum} received, initiating shutdown.")
    global daemon_running
    daemon_running = False

    # Attempt to stop the Flask server
    # (If running under systemd or a non-Werkzeug server, it might just exit the main thread.)
    try:
        shutdown_server()
    except Exception as e:
        logger.debug(f"Exception while shutting down Flask: {e}")

    # If running in the foreground, user can also press Ctrl+C again, but let's exit gracefully
    sys.exit(0)

In [8]:
###############################################################################
# ARGUMENT PARSING
###############################################################################
def parse_args():

    # detect jupyter's ipykernel_launcher and trim the jupyter args
    if 'ipykernel_launcher' in sys.argv[0]:
        argv = sys.argv[3:]
    else:
        argv = sys.argv
        
    parser = argparse.ArgumentParser(description="PaperPi App")
    parser.add_argument("-d", "--daemon", action="store_true",
                        help="Run in daemon mode (use system-wide config)")

    parser.add_argument("-c", "--config", type=str, default=None,
                         help="Path to application configuration yaml file")

    parser.add_argument("-p", "--plugin_config", type=str, default=None,
                          help="Path to plugin configuration yaml file")
    
    return parser.parse_args(argv)

In [55]:
def cleanup(msg: str = None):
    if msg:
        print(msg)

    sys.exit(0)

In [56]:
###############################################################################
# MAIN ENTRY POINT
###############################################################################

def main():
    # Register our signal handlers
    signal.signal(signal.SIGINT, handle_signal)
    signal.signal(signal.SIGTERM, handle_signal)

    args = parse_args()
    
    if running_under_systemd() or args.daemon:
        # configuration file in daemon mode
        file_app_config = PATH_DAEMON_CONFIG / FNAME_APPLICATION_CONFIG
    else:
        # configuration in user on-demand mode
        file_app_config = PATH_USER_CONFIG / FNAME_APPLICATION_CONFIG
    
    # apply override from command line
    if args.config:
        file_app_config = Path(args.config)
    
    
    # get the parent dir of the application configuration file 
    path_app_config = file_app_config.parent
    
    # use the supplied plugin_config_file
    if args.plugin_config:
        file_plugin_config = Path(args.plugin_config)
    # otherwise use the default
    else:
        file_plugin_config = path_app_config / FNAME_PLUGIN_CONFIG
    
    file_app_schema = PATH_APP_CONFIG / FNAME_APPLICATION_SCHEMA

    try:
        app_config_yaml = load_yaml_file(file_app_config)
        config_schema_yaml = load_yaml_file(file_app_schema)

        # add plugin schema and plugin manager schema here
        
    except (FileNotFoundError, ValueError) as e:
        logger.error(f'Failed to read configuration files: {e}')
        cleanup()
        
 
    try:
        logger.info('Validating application configuration')
        logger.debug(file_app_config)
        app_configuration = validate_config(app_config_yaml[KEY_APPLICATION_SCHEMA], config_schema_yaml)
    except ValueError as e:
        logger.error(f'Failed to validate configuration in {file_app_config}')
        cleanup()
    
    
    # get the web port and log level
    web_port = app_configuration.get('port', PORT)
    log_level = app_configuration.get('log_level', logging.WARNING)
    
    
    # get the resolution & screenmode from the configured epaper driver

    ### hard coded for the moment
    resolution = (800, 640)
    screen_mode = 'L'
    
    app_configuration['resolution'] = resolution
    app_configuration['screen_mode'] = screen_mode
    
    
    # load the plugin configuration; validation will happen in the plugin manager
    plugin_configuration = load_yaml_file(file_plugin_config)
    
    # build the plugin manager 
    plugin_manager = PluginManager()
    
    plugin_manager.plugin_path = PATH_APP_PLUGINS
    plugin_manager.config_path = PATH_APP_CONFIG
    plugin_manager.base_schema_file = FNAME_PLUGIN_MANAGER_SCHEMA
    plugin_manager.plugin_schema_file = FNAME_PLUGIN_SCHEMA
    try:
        plugin_manager.config = app_configuration
    except ValueError as e:
        msg = f"Configuration file error: {e}"
        logger.error(msg)
        # do something to bail out and stop loading here
        
    # add the plugins based on the loaded configurations
    plugin_manager.add_plugins(plugin_configuration[KEY_PLUGIN_DICT])
    # validate and load the plugins
    plugin_manager.load_plugins()



    
    
    # # Start the daemon loop in a background thread
    # thread = threading.Thread(target=daemon_loop, daemon=True)
    # thread.start()

    # # Start Flask in the main thread (blocking call)
    # logger.info(f"Starting Flask on port {web_port}...")
    # # In production behind systemd, you might switch to gunicorn or uwsgi; for dev, this is fine.
    # app.run(host="0.0.0.0", port=PORT, debug=False)

In [67]:
test_args = [
             # ('-d', None), 
             ('-c', '~/.config/com.txoof.paperpi/paperpi_config.yaml'), 
             # ('-p', '~/.config/com.txoof.paperpi/plugins_config.yaml')
            ]

for key, value in test_args:
    try:
        idx = sys.argv.index(key)
        if value is not None:
            # Check if the next argument exists and update it
            if idx + 1 < len(sys.argv):
                sys.argv[idx + 1] = value
            else:
                # If no value exists, append it
                sys.argv.append(value)
    except ValueError:
        # If key is not in sys.argv, add it along with the value (if applicable)
        sys.argv.append(key)
        if value is not None:
            sys.argv.append(value)
        
print(sys.argv) 

['/home/pi/src/PaperPi-Web/PaperPi-Web-venv-33529be2c6/lib/python3.11/site-packages/ipykernel_launcher.py', '-f', '/home/pi/.local/share/jupyter/runtime/kernel-8e90620b-2d11-43bc-af0a-c5db962e2432.json', '-c', '~/.config/com.txoof.paperpi/paperpi_config.yaml']


In [68]:
sys.argv

['/home/pi/src/PaperPi-Web/PaperPi-Web-venv-33529be2c6/lib/python3.11/site-packages/ipykernel_launcher.py',
 '-f',
 '/home/pi/.local/share/jupyter/runtime/kernel-8e90620b-2d11-43bc-af0a-c5db962e2432.json',
 '-c',
 '~/.config/com.txoof.paperpi/paperpi_config.yaml']

In [69]:
if __name__ == "__main__":
    main()

2025-04-13 19:39:42 [INFO] Reading yaml file at /home/pi/.config/com.txoof.paperpi/paperpi_config.yaml
2025-04-13 19:39:42 [INFO] YAML file '/home/pi/.config/com.txoof.paperpi/paperpi_config.yaml' loaded successfully.
2025-04-13 19:39:42 [INFO] Reading yaml file at /home/pi/src/PaperPi-Web/config/paperpi_config_schema.yaml
2025-04-13 19:39:42 [INFO] YAML file '/home/pi/src/PaperPi-Web/config/paperpi_config_schema.yaml' loaded successfully.
2025-04-13 19:39:42 [INFO] Validating application configuration
2025-04-13 19:39:42 [INFO] Configuration validated successfully.
2025-04-13 19:39:42 [INFO] Reading yaml file at /home/pi/.config/com.txoof.paperpi/paperpi_plugins.yaml
2025-04-13 19:39:42 [INFO] YAML file '/home/pi/.config/com.txoof.paperpi/paperpi_plugins.yaml' loaded successfully.
2025-04-13 19:39:42 [INFO] PluginManager initialized.
2025-04-13 19:39:42 [INFO] Schema '/home/pi/src/PaperPi-Web/config/plugin_manager_schema.yaml' cached successfully.
2025-04-13 19:39:42 [INFO] Schema '/h